In [5]:
# In this scenario we are going to grab the info from excel files.
'''
We need:
1. Prices in different currencies
2. List with Pusblisher names.
3. Clean Publisher Name
4. Check Suppliers, currencies, publishers, discount, shipping costs
5. Check Suppliers special discounts
6. Check restrictions
7. Get landed Cost
8. List of ISBN to ananlyse.

'''

'\nWe need:\n1. Prices in different currencies\n2. List with Pusblisher names.\n3. Clean Publisher Name\n4. Check Suppliers, currencies, publishers, discount, shipping costs\n5. Check Suppliers special discounts\n6. Check restrictions\n7. Get landed Cost\n8. List of ISBN to ananlyse.\n\n'

In [7]:
# 0. Import Libraries and credentials
import pandas as pd
import pyodbc
import numpy as np
import sqlCredentials as sql

In [9]:
# 0. Get databse connection
# Let's pick all the titles gwe can get from the pricing matrix.

prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

prouc_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

In [11]:
#1. Prices at different currencies
#this time we are going to use all the titles
list_price_sql = """
SELECT *
FROM Isbn.ListPrice
"""

list_price = pd.read_sql(list_price_sql, prod_db)
list_price.head()

,Id,Isbn,Currency,Price,PriceSource,PriceLastRefreshed,Timestamp
0,1,9780061706585,USD,9.99,GoldenList,2020-03-29 18:00:01.3733333,2020-03-29 19:03:37.883333
1,2,9780061708718,USD,16.99,GoldenList,2020-03-29 18:00:01.3733333,2020-03-29 19:03:37.883333
2,3,9780061708763,USD,16.99,GoldenList,2020-03-29 18:00:01.3733333,2020-03-29 19:03:37.883333
3,4,9780061709562,USD,8.99,GoldenList,2020-03-29 18:00:01.3733333,2020-03-29 19:03:37.883333
4,5,9780061710308,USD,19.99,GoldenList,2020-03-29 18:00:01.3733333,2020-03-29 19:03:37.883333


In [13]:
list_price.shape

(1748970, 7)

In [15]:
list_price['PriceSource'].unique()

array(['GoldenList', 'BookDepotInventory', 'GardnersInventory',
       'BertramsInventory', 'BowkerNepModel', 'NielsenFtp',
       'TitlePageScraper', 'BowkerApiModel', 'SAPPurchaseOrder',
       'PearsonCanadaScraperModel', 'AbePolandScraper'], dtype=object)

In [17]:
list_price.drop(columns = ['Id', 'PriceSource', 'PriceLastRefreshed', 'Timestamp'], inplace = True)
list_price.head()

,Isbn,Currency,Price
0,9780061706585,USD,9.99
1,9780061708718,USD,16.99
2,9780061708763,USD,16.99
3,9780061709562,USD,8.99
4,9780061710308,USD,19.99


In [12]:
bilblo_sql = """
SELECT *
FROM Isbn.Bibliography
"""
biblio = pd.read_sql(bilblo_sql, prod_db)
biblio.head()

,Isbn,Title,TitleSource,TitleLastRefreshed,Publisher,PublisherSource,PublisherLastRefreshed,Author,AuthorSource,AuthorLastRefreshed,PublicationDate,PublicationDateSource,PublicationDateLastRefreshed,Copyright,CopyrightSource,CopyrightLastRefreshed,Binding,BindingSource,BindingLastRefreshed,Timestamp
0,9780000002402,None,None,None,LWW,GoldenList,2020-03-29 18:00:01.3733333,None,None,None,None,None,None,None,None,None,None,None,None,2020-03-29 19:02:44.370
1,9780000002679,None,None,None,LWW,GoldenList,2020-03-29 18:00:01.3733333,None,None,None,None,None,None,None,None,None,None,None,None,2020-03-29 19:02:44.370
2,9780000003102,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2020-03-29 19:02:44.370
3,9780000003201,None,None,None,LWW,GoldenList,2020-03-29 18:00:01.3733333,None,None,None,None,None,None,None,None,None,None,None,None,2020-03-29 19:02:44.370
4,9780000006493,None,None,None,LWW,GoldenList,2020-03-29 18:00:01.3733333,None,None,None,None,None,None,None,None,None,None,None,None,2020-03-29 19:02:44.370


In [14]:
sup_list = pd.merge(list_price, biblio, how='left', on = 'Isbn' )

In [16]:
sup_list.head()

,Id,Isbn,Currency,Price,PriceSource,PriceLastRefreshed,Timestamp_x,Title,TitleSource,TitleLastRefreshed,...,PublicationDate,PublicationDateSource,PublicationDateLastRefreshed,Copyright,CopyrightSource,CopyrightLastRefreshed,Binding,BindingSource,BindingLastRefreshed,Timestamp_y
0,1,9780061706585,USD,9.99,GoldenList,2020-03-29 18:00:01.3733333,2020-03-29 19:03:37.883333,None,None,None,...,None,None,None,None,None,None,None,None,None,2020-03-29 19:02:44.370
1,2,9780061708718,USD,16.99,GoldenList,2020-03-29 18:00:01.3733333,2020-03-29 19:03:37.883333,None,None,None,...,None,None,None,None,None,None,None,None,None,2020-03-29 19:02:44.370
2,3,9780061708763,USD,16.99,GoldenList,2020-03-29 18:00:01.3733333,2020-03-29 19:03:37.883333,None,None,None,...,None,None,None,None,None,None,None,None,None,2020-03-29 19:02:44.370
3,4,9780061709562,USD,8.99,GoldenList,2020-03-29 18:00:01.3733333,2020-03-29 19:03:37.883333,None,None,None,...,None,None,None,None,None,None,None,None,None,2020-03-29 19:02:44.370
4,5,9780061710308,USD,19.99,GoldenList,2020-03-29 18:00:01.3733333,2020-03-29 19:03:37.883333,None,None,None,...,None,None,None,None,None,None,None,None,None,2020-03-29 19:02:44.370


In [18]:
sup_list.columns

Index(['Id', 'Isbn', 'Currency', 'Price', 'PriceSource', 'PriceLastRefreshed',
       'Timestamp_x', 'Title', 'TitleSource', 'TitleLastRefreshed',
       'Publisher', 'PublisherSource', 'PublisherLastRefreshed', 'Author',
       'AuthorSource', 'AuthorLastRefreshed', 'PublicationDate',
       'PublicationDateSource', 'PublicationDateLastRefreshed', 'Copyright',
       'CopyrightSource', 'CopyrightLastRefreshed', 'Binding', 'BindingSource',
       'BindingLastRefreshed', 'Timestamp_y'],
      dtype='object')

In [331]:
sup_list = sup_list[['Isbn','Currency', 'Price','Title', 'Publisher', 'Author']]

In [332]:
sup_list.head()

,Isbn,Currency,Price,Title,Publisher,Author
0,9780061706585,USD,9.99,None,HARPER COLLINS,None
1,9780061708718,USD,16.99,None,HARPER COLLINS,None
2,9780061708763,USD,16.99,None,HARPER COLLINS,None
3,9780061709562,USD,8.99,None,HARPER COLLINS,None
4,9780061710308,USD,19.99,None,HARPER COLLINS,None


In [333]:
sup_list['Publisher'].unique()

array(['HARPER COLLINS', 'Greenwillow Books', None, ..., 'Plectica LLC',
       'Detritus Books', 'Hallen House Publishing'], dtype=object)

In [334]:
sup_list = sup_list.fillna('N/A')
sup_list.isna().sum()

Isbn         0
Currency     0
Price        0
Title        0
Publisher    0
Author       0
dtype: int64

In [335]:
# 3. Clean publisher
dictio_sql = """
SELECT *
FROM dbo.PublisherDictionary
"""

dictio = pd.read_sql(dictio_sql, prouc_db)
dictio.head()

,PublisherLong,PublisherShort
0,"""Abrams, Inc.""",ABRAMS
1,"""Addison-Wesley Longman, Incorporated""",PEARSON
2,"""Addison-Wesley Longman, Limited""",PEARSON
3,"""Allyn & Bacon, Incorporated""",PEARSON
4,"""American Occupational Therapy Association, In...","American Occupational Therapy Association, Inc..."


In [336]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

In [337]:
for col in list(dictio.columns):
    dictio[col] = dictio.apply(lambda x: clean_up(x[col]), axis =1)

In [338]:
dictio.head()

,PublisherLong,PublisherShort
0,"""ABRAMS, INC.""",ABRAMS
1,"""ADDISON-WESLEY LONGMAN, INCORPORATED""",PEARSON
2,"""ADDISON-WESLEY LONGMAN, LIMITED""",PEARSON
3,"""ALLYN & BACON, INCORPORATED""",PEARSON
4,"""AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, IN...","AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, INC..."


In [339]:
for col in list(sup_list.drop(columns = 'Price').columns):
    sup_list[col] = sup_list.apply(lambda x: clean_up(x[col]), axis =1)

In [340]:
sup_list.head()

,Isbn,Currency,Price,Title,Publisher,Author
0,9780061706585,USD,9.99,N/A,HARPER COLLINS,N/A
1,9780061708718,USD,16.99,N/A,HARPER COLLINS,N/A
2,9780061708763,USD,16.99,N/A,HARPER COLLINS,N/A
3,9780061709562,USD,8.99,N/A,HARPER COLLINS,N/A
4,9780061710308,USD,19.99,N/A,HARPER COLLINS,N/A


In [341]:
# Merge on Publisher
sup_list2 = pd.merge(sup_list, dictio, how='left', left_on = 'Publisher', right_on = 'PublisherLong' )

In [342]:
sup_list2.head()

,Isbn,Currency,Price,Title,Publisher,Author,PublisherLong,PublisherShort
0,9780061706585,USD,9.99,N/A,HARPER COLLINS,N/A,HARPER COLLINS,HARPER COLLINS
1,9780061708718,USD,16.99,N/A,HARPER COLLINS,N/A,HARPER COLLINS,HARPER COLLINS
2,9780061708763,USD,16.99,N/A,HARPER COLLINS,N/A,HARPER COLLINS,HARPER COLLINS
3,9780061709562,USD,8.99,N/A,HARPER COLLINS,N/A,HARPER COLLINS,HARPER COLLINS
4,9780061710308,USD,19.99,N/A,HARPER COLLINS,N/A,HARPER COLLINS,HARPER COLLINS


In [343]:
# 4. get suppliers discount
sup_pub = pd.read_excel('suppliers.xlsx',  sheet_name='pub')
sup_pub.head()

,Supplier,Publisher,Currency,Discount,Shipping Cost,max quantity,Min,Region
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,0,AUSTRALIA
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,0,AUSTRALIA
2,COINFO,AIAA,AUD,0.30,3.3,10000,0,AUSTRALIA
3,COINFO,ALIEN & UNWIN,AUD,0.33,3.3,10000,0,AUSTRALIA
4,COINFO,Amer Psych Ass.,AUD,0.33,3.3,10000,0,AUSTRALIA


In [344]:
for col in ['Supplier', 'Publisher', 'Currency', 'Region']:
    sup_pub[col] = sup_pub.apply(lambda x: clean_up(x[col]), axis =1)

In [345]:
sup_list3 = pd.merge(sup_list2, sup_pub, how='inner', left_on = ['Currency', 'PublisherShort'], right_on = ['Currency', 'Publisher'] )

In [346]:
sup_list3

,Isbn,Currency,Price,Title,Publisher_x,Author,PublisherLong,PublisherShort,Supplier,Publisher_y,Discount,Shipping Cost,max quantity,Min,Region
0,9780000002402,GBP,306.00,N/A,LWW,N/A,LWW,LWW,ABE,LWW,0.55,3.47,10000,0,EUROPE
1,9780000006493,GBP,772.00,N/A,LWW,N/A,LWW,LWW,ABE,LWW,0.55,3.47,10000,0,EUROPE
2,9780000009883,GBP,92.00,N/A,LWW,N/A,LWW,LWW,ABE,LWW,0.55,3.47,10000,0,EUROPE
3,9780000003201,GBP,265.00,N/A,LWW,N/A,LWW,LWW,ABE,LWW,0.55,3.47,10000,0,EUROPE
4,9780000002679,GBP,425.00,N/A,LWW,N/A,LWW,LWW,ABE,LWW,0.55,3.47,10000,0,EUROPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485889,9789027252722,AUD,78.95,"PIDGINS, CREOLES AND MIXED LANGUAGES: AN INTRO...",JOHN BENJAMINS PUBLISHING,VIVEKA VELUPILLAI,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
485890,9789027212092,AUD,79.95,HISTORY OF THE ENGLISH LANGUAGE,JOHN BENJAMINS PUBLISHING,ELLY VAN GELDEREN,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
485891,9789027213273,AUD,78.95,PRONUNCIATION FUNDAMENTALS,JOHN BENJAMINS PUBLISHING,TRACEY M. DERWING; MURRAY J. MUNRO,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
485892,9789027219060,AUD,75.95,COGNITIVE EXPLORATION OF LANGUAGE AND LINGUISTICS,JOHN BENJAMINS PUBLISHING,RENE DIRVEN,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA


In [347]:
sup_list3.drop(columns = ['Publisher_x', 'PublisherLong', 'Publisher_y'], inplace = True)

In [348]:
sup_list3.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region
0,9780000002402,GBP,306.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE
1,9780000006493,GBP,772.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE
2,9780000009883,GBP,92.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE
3,9780000003201,GBP,265.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE
4,9780000002679,GBP,425.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE


In [349]:
# 5. Check Suppliers special discounts
sp_disc = pd.read_excel("../../../../Vendors/. Short Discounted/all_short_discount_list.xlsx")

In [350]:
sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [351]:
sp_disc.dtypes

SUPPLIER     object
ISBN          int64
DISCOUNT    float64
dtype: object

In [352]:
sp_disc['ISBN'] = sp_disc['ISBN'].astype(str)
sp_disc.dtypes

SUPPLIER     object
ISBN         object
DISCOUNT    float64
dtype: object

In [353]:
for col in ['SUPPLIER', 'ISBN']:
    sp_disc[col] = sp_disc.apply(lambda x: clean_up(x[col]), axis =1)

sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [354]:
sup_list4 = pd.merge(sup_list3, sp_disc, how='left', left_on = ['Supplier', 'Isbn'], right_on = ['SUPPLIER', 'ISBN'] )

In [355]:
sup_list4.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT
0,9780000002402,GBP,306.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
1,9780000006493,GBP,772.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
2,9780000009883,GBP,92.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
3,9780000003201,GBP,265.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
4,9780000002679,GBP,425.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN


In [356]:
sup_list4[sup_list4['SUPPLIER']=='ABE']

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT
51,9780781771696,GBP,55.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781771696,0.35
64,9780781768702,GBP,55.00,ACSM'S RESOURCES FOR CLINICAL EXERCISE PHYSIOLOGY,AMERICAN COLLEGE OF SPORTS MEDICINE,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781768702,0.35
79,9780781777988,GBP,58.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781777988,0.35
83,9780781782678,GBP,72.00,ACSM'S FOUNDATIONS OF STRENGTH TRAINING AND CO...,AMERICAN COLLEGE OF SPORTS MEDICINE (ACSM) STA...,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781782678,0.35
148,9780781787154,GBP,58.00,STEP-BY-STEP MASSAGE THERAPY PROTOCOLS FOR COM...,CHARLOTTE MICHAEL VERSAGI,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781787154,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482712,9781975102012,GBP,40.99,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9781975102012,0.35
482714,9781975117337,GBP,150.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9781975117337,0.35
482719,9781975122676,GBP,240.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9781975122676,0.35
482720,9781975123697,GBP,270.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9781975123697,0.35


In [357]:
sup_list4.isna().sum()

Isbn                   0
Currency               0
Price                  0
Title                  0
Author                 0
PublisherShort         0
Supplier               0
Discount               0
Shipping Cost          0
max quantity           0
Min                    0
Region                 0
SUPPLIER          484414
ISBN              484414
DISCOUNT          484414
dtype: int64

In [358]:
sup_list4['final_discount'] = np.where(sup_list4['DISCOUNT'].isna(), sup_list4['Discount'],sup_list4['DISCOUNT'])

In [359]:
sup_list4.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT,final_discount
0,9780000002402,GBP,306.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
1,9780000006493,GBP,772.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
2,9780000009883,GBP,92.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
3,9780000003201,GBP,265.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
4,9780000002679,GBP,425.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55


In [360]:
sup_list4[sup_list4['Isbn'] == '9780781762021']

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT,final_discount
269,9780781762021,GBP,57.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781762021,0.35,0.35


In [361]:
sup_list4 = sup_list4[['Isbn','Title', 'Author', 'PublisherShort', 'Supplier','Currency', 'Price', 'final_discount' ,'Shipping Cost', 'max quantity', 'Min', 'Region']]

In [362]:
sup_list4.head()

,Isbn,Title,Author,PublisherShort,Supplier,Currency,Price,final_discount,Shipping Cost,max quantity,Min,Region
0,9780000002402,N/A,N/A,LWW,ABE,GBP,306.0,0.55,3.47,10000,0,EUROPE
1,9780000006493,N/A,N/A,LWW,ABE,GBP,772.0,0.55,3.47,10000,0,EUROPE
2,9780000009883,N/A,N/A,LWW,ABE,GBP,92.0,0.55,3.47,10000,0,EUROPE
3,9780000003201,N/A,N/A,LWW,ABE,GBP,265.0,0.55,3.47,10000,0,EUROPE
4,9780000002679,N/A,N/A,LWW,ABE,GBP,425.0,0.55,3.47,10000,0,EUROPE


In [363]:
sup_list4.columns = map(str.lower, sup_list4.columns)

In [364]:
sup_list4.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
0,9780000002402,N/A,N/A,LWW,ABE,GBP,306.0,0.55,3.47,10000,0,EUROPE
1,9780000006493,N/A,N/A,LWW,ABE,GBP,772.0,0.55,3.47,10000,0,EUROPE
2,9780000009883,N/A,N/A,LWW,ABE,GBP,92.0,0.55,3.47,10000,0,EUROPE
3,9780000003201,N/A,N/A,LWW,ABE,GBP,265.0,0.55,3.47,10000,0,EUROPE
4,9780000002679,N/A,N/A,LWW,ABE,GBP,425.0,0.55,3.47,10000,0,EUROPE


In [365]:
# Add western campus
# Shipping Cost = 2
# 0-150CAD 10% off
# 150-200CAD 12% off
# > 200CAD 14%

can_sup = sup_list2[sup_list2['Currency'] == 'CND']
can_sup.columns = map(str.lower, can_sup.columns)
can_sup['supplier'] = 'WC'
can_sup['final_discount'] = np.where(can_sup['price']<150, 0.1, np.where((can_sup['price'] >=150) & (can_sup['price']<200),0.12,0.14))
can_sup['max quantity'] = 10000
can_sup['region'] = 'NORTH AMERICA'
can_sup['shipping cost'] = 2
can_sup['min'] = 0
can_sup.head()

,isbn,currency,price,title,publisher,author,publisherlong,publishershort,supplier,final_discount,max quantity,region,shipping cost,min
120,9780008117320,CND,40.00,N/A,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
131,9780008121716,CND,16.99,N/A,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
206,9780008135904,CND,9.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
211,9780008135935,CND,7.99,COLLINS PORTUGUESE PHRASEBOOK AND DICTIONARY G...,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
219,9780008136628,CND,28.99,COLLINS WORLD ATLAS: ILLUSTRATED EDITION,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0


In [366]:
can_sup = can_sup[['isbn', 'title', 'author', 'publishershort', 'supplier','currency','price','final_discount', 'shipping cost', 'max quantity', 'min', 'region']]

In [367]:
can_sup.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
120,9780008117320,N/A,N/A,N/A,WC,CND,40.00,0.1,2,10000,0,NORTH AMERICA
131,9780008121716,N/A,N/A,N/A,WC,CND,16.99,0.1,2,10000,0,NORTH AMERICA
206,9780008135904,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,WC,CND,9.99,0.1,2,10000,0,NORTH AMERICA
211,9780008135935,COLLINS PORTUGUESE PHRASEBOOK AND DICTIONARY G...,N/A,N/A,WC,CND,7.99,0.1,2,10000,0,NORTH AMERICA
219,9780008136628,COLLINS WORLD ATLAS: ILLUSTRATED EDITION,N/A,N/A,WC,CND,28.99,0.1,2,10000,0,NORTH AMERICA


In [368]:
sup_list5 = sup_list4.append(can_sup, ignore_index= True)
sup_list5.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
0,9780000002402,N/A,N/A,LWW,ABE,GBP,306.0,0.55,3.47,10000,0,EUROPE
1,9780000006493,N/A,N/A,LWW,ABE,GBP,772.0,0.55,3.47,10000,0,EUROPE
2,9780000009883,N/A,N/A,LWW,ABE,GBP,92.0,0.55,3.47,10000,0,EUROPE
3,9780000003201,N/A,N/A,LWW,ABE,GBP,265.0,0.55,3.47,10000,0,EUROPE
4,9780000002679,N/A,N/A,LWW,ABE,GBP,425.0,0.55,3.47,10000,0,EUROPE


In [369]:
sup_list5['currency'].unique()

array(['GBP', 'USD', 'AUD', 'CND'], dtype=object)

In [370]:
# 5. Check Suppliers special discounts
rest = pd.read_excel("../../../../Vendors/. Restrictions\All Restricted List.xlsx")
rest.head()

,ISBN13,Vendor,Publisher,Comment,Vendor&ISBN,date added,Clear to order
0,9780323567428,Risky,NaN,COINFO: Dosage Error,Risky9780323567428,NaT,NaN
1,9780500292105,Risky,NaN,NaN,Risky9780500292105,NaT,NaN
2,9781936523443,Risky,NaN,NaN,Risky9781936523443,NaT,NaN
3,9780134149530,Risky,NaN,NaN,Risky9780134149530,NaT,NaN
4,9780134205588,Risky,NaN,NaN,Risky9780134205588,NaT,NaN


In [371]:
rest.columns = map(str.lower, rest.columns)
rest.head()

,isbn13,vendor,publisher,comment,vendor&isbn,date added,clear to order
0,9780323567428,Risky,NaN,COINFO: Dosage Error,Risky9780323567428,NaT,NaN
1,9780500292105,Risky,NaN,NaN,Risky9780500292105,NaT,NaN
2,9781936523443,Risky,NaN,NaN,Risky9781936523443,NaT,NaN
3,9780134149530,Risky,NaN,NaN,Risky9780134149530,NaT,NaN
4,9780134205588,Risky,NaN,NaN,Risky9780134205588,NaT,NaN


In [374]:
rest.dtypes

isbn13                    object
vendor                    object
publisher                 object
comment                   object
vendor&isbn               object
date added        datetime64[ns]
clear to order            object
dtype: object

In [376]:
rest['isbn13'] = rest['isbn13'].astype(str)

In [377]:
for col in ['isbn13', 'vendor']:
    rest[col] = rest.apply(lambda x: clean_up(x[col]), axis =1)

rest = rest[['isbn13','vendor']]
rest.head()

,isbn13,vendor
0,9780323567428,RISKY
1,9780500292105,RISKY
2,9781936523443,RISKY
3,9780134149530,RISKY
4,9780134205588,RISKY


In [380]:
rest.shape

(88613, 2)

In [379]:
risky = rest[rest['vendor'] == 'RISKY']
risky.head()

,isbn13,vendor
0,9780323567428,RISKY
1,9780500292105,RISKY
2,9781936523443,RISKY
3,9780134149530,RISKY
4,9780134205588,RISKY


In [381]:
risky.shape

(486, 2)

In [ ]:
sup_list5.to_csv('suppliers_excel.csv', index= False)